In [17]:
%reset -f

In [18]:
#ativar a venv -> .\.venv\Scripts\Activate.ps1
#desativar a venv -> deactivate
import pandas as pd 
import numpy as np 
from pathlib import Path

bases_path = Path("bases_B")
output =  Path("OUTPUT")
output.mkdir(exist_ok=True)

arquivo_listagem = bases_path / "listagem.xlsx"
arquivo_checklist = bases_path / "HERING_FRANQUIA_checklist_RA_RANQUIA_2025-12-17.xlsx"
fases = bases_path / "fase.xlsx"

df_listagem = pd.read_excel(arquivo_listagem, sheet_name='Lojas FRQ', engine="calamine")
df_checklist = pd.read_excel(arquivo_checklist, engine="calamine")
df_fases = pd.read_excel(fases, engine="calamine")


In [19]:
df_checklist.head()

,CANAL,FILIAL,FILIAL_NOME,REGIAO,SKU,ARTIGO_COR,PRODUTO,DESC_PRODUTO,COR,TAM,...,DATA,CNPJ,CLISAP,CONT_RUPTURA,CONT_RUPTURA_RESSUP,CONT_FALTA,VOLUME_FALTA,CONT_ALVO,CONT_EXCESSO,VOLUME_EXCESSO
0,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-N10SI-M,H4FA-N10SI,H4FA,BERMUDA MASC,N10SI,M,...,2025-12-17,51138625000317,121100,0,0,0,0,0,1,5
1,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-XXG,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,XXG,...,2025-12-17,51138625000317,121100,0,0,0,0,0,1,1
2,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-XG,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,XG,...,2025-12-17,51138625000317,121100,0,0,0,0,1,0,0
3,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-P,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,P,...,2025-12-17,51138625000317,121100,0,0,0,0,1,0,0
4,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-M,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,M,...,2025-12-17,51138625000317,121100,0,0,0,0,1,0,0


In [20]:
df_checklist.columns

Index(['CANAL', 'FILIAL', 'FILIAL_NOME', 'REGIAO', 'SKU', 'ARTIGO_COR',
       'PRODUTO', 'DESC_PRODUTO', 'COR', 'TAM', 'TAM_GRADE', 'NEGOCIO',
       'PIRAMIDE', 'GENERO', 'IDADE_PRODUTO', 'GRUPO_PRODUTO', 'LINHA',
       'COLECAO', 'VELOCIDADE_VENDA', 'LEADTIME', 'COBERTURA', 'MINIMO',
       'PRESENTE', 'LOJA', 'TRANSITO', 'EMBALADO', 'RESERVADO', 'REGULADOR',
       'EM_MOVIMENTO', 'VENDAS_D_14', 'VENDAS_D_1', 'ALVO', 'NECESSIDADE',
       'RESSUPRIR', 'EST_TOTAL', 'EST_DISP', 'CATEGORIA_EST_TOTAL',
       'CATEGORIA_EST_DISP', 'DATA', 'CNPJ', 'CLISAP', 'CONT_RUPTURA',
       'CONT_RUPTURA_RESSUP', 'CONT_FALTA', 'VOLUME_FALTA', 'CONT_ALVO',
       'CONT_EXCESSO', 'VOLUME_EXCESSO'],
      dtype='object')

In [21]:
df_listagem.head()

,CNPJ,FILIAL,Nome loja,Pot. Venda,UF,Cidade,Região,Franqueado,Ativo R.A,Negócio,Sortimento Ajustado
0,71868467000104,3,HERING ALAMEDA BARROS,P,SP,SAO PAULO,Sudeste,AURIDENICHE STEFANO,SIM,HA+HI,P_HA+HI
1,37069482001703,5,HS FREI CANECA,P,SP,SAO PAULO,Sudeste,ARLINDO DE OLIVEIRA LIMA FILHO,SIM,HA+HI,P_HA+HIRED
2,9420677000153,7,HERING PAMPLONA,P,SP,SAO PAULO,Sudeste,MARCO ANTONIO SIMOES CHADAD,SIM,HA+HI,P_HA+HIRED
3,11850562001627,9,RUA AUGUSTA,P,SP,SAO PAULO,Sudeste,ANDRE HERING,SIM,HA+HI+HK,P_HA+HI+HK
4,7057579000178,10,HERING ACLIMACAO,G,SP,SAO PAULO,Sudeste,LUIZ ANTONIO LUCIANO,SIM,HA+HI+HK,G_HA+HI+HK


In [22]:
df_listagem.columns

Index(['CNPJ', 'FILIAL', 'Nome loja', 'Pot. Venda', 'UF', 'Cidade', 'Região',
       'Franqueado', 'Ativo R.A', 'Negócio', 'Sortimento Ajustado'],
      dtype='object')

In [23]:
df_fases.head()

,Artigo Cor,Negócio,Grupo,fase
0,0111AX7EN,HERING ADULTO - MASCULINO,REGATAS,2ª Fase
1,0111M2H07S,HERING ADULTO - MASCULINO,REGATAS,2ª Fase
2,0111MD307S,HERING ADULTO - MASCULINO,REGATAS,2ª Fase
3,0111N0A00S,HERING ADULTO - MASCULINO,REGATAS,2ª Fase
4,0111N1007S,HERING ADULTO - MASCULINO,REGATAS,2ª Fase


In [24]:
def validar_checklist(df_listagem, df_checklist, df_fases):
    
   #criar uma coluna uxiliar de alvo calculado = velocidade de venda* (leadtime + cobertura) com duas cada decimais
    df_checklist['Alvo Calculado'] = df_checklist['VELOCIDADE_VENDA'] * (df_checklist['LEADTIME'] + df_checklist['COBERTURA'])
    df_checklist['Alvo Calculado'] = df_checklist['Alvo Calculado'].round(2)
    # alvo final = pegar o máximo entre a coluna alvo calculado e a coluna alvo minimo e arredondar para cima o valor máximo entre os dois
    df_checklist['Alvo Final'] = np.maximum(df_checklist['Alvo Calculado'], df_checklist['MINIMO']).apply(np.ceil) 
   # validar se a coluna alvo final é igual a coluna ALVO, se sim retornar sim senão não
    df_checklist["Validação Alvo"] = np.where(df_checklist["Alvo Final"] == df_checklist["ALVO"], 'SIM', 'NÃO')
     # concatenar a coluna PRODUTO  e a coluna COR
    df_checklist['Artigo Cor'] = df_checklist['PRODUTO'].astype(str)  + df_checklist['COR'].astype(str)
    # validar a coluna MINIMO se é maior que coluna alvo calculado, se sim retornar sim senão não
    df_checklist['Validação MINIMO'] = np.where(df_checklist['MINIMO'] > df_checklist['Alvo Calculado'], 'SIM', 'NÃO')
    #validar se a coluna NECESSIDADE é maior ou igual a coluna de RESSUPRIR
    df_checklist['Validação RESSUPRIR'] = np.where(df_checklist['NECESSIDADE'] >= df_checklist['RESSUPRIR'], 'SIM', 'NÃO')
    # fazer um merge entre o df_checklist e o df_listagem pegar as colunas Franqueado, Pot.Venda, Sortimento
    df_checklist = df_checklist.merge(df_listagem[['FILIAL', 'Franqueado', 'Pot. Venda', 'Sortimento Ajustado']], on='FILIAL', how='left')
    #Se o sortimento não for encontrado retorne 0
    df_checklist['Sortimento Ajustado'] = df_checklist['Sortimento Ajustado'].fillna(0)
    #validar a coluna NECESSIDADE se é maior que 0
    df_checklist['NECESSIDADE ?'] = np.where(df_checklist['NECESSIDADE'] > 0, 'SIM', 'NÃO')
    #validar se a coluna NECESSIDADE é igual a se a coluna ALVO - RESERVADO - EMBALOADO for menor que 0 é 0, senão ALVO - RESERVADO - EMBALOADO E COMPARAR COM A COLUNA NECESSIDADE e se for igual retorne sim senão não
    necessidade_calculada = df_checklist['ALVO'] - df_checklist['PRESENTE'] - df_checklist['EMBALADO'] - df_checklist['RESERVADO']
    necessidade_calculada = necessidade_calculada = np.maximum(0, necessidade_calculada)
    df_checklist['Validação NECESSIDADE'] = np.where(df_checklist['NECESSIDADE'] == necessidade_calculada, 'SIM', 'NÃO')
    # concatenar a coluna FILIAL + - + ARTIGO_COR
    df_checklist['Filial-Artigo_Cor'] = df_checklist['FILIAL'].astype(str) + ' - ' + df_checklist['ARTIGO_COR'].astype(str)
    # fazer um merge entre o df_checklist e o df_fases e pegar a coluna fase
    df_checklist = df_checklist.merge(df_fases[['Artigo Cor', ' fase']], on='Artigo Cor', how='left')

    return  df_checklist # Retornar True se todas as validações passarem

In [25]:
#exibir e chamar a função validar_checklist
df_resultado = validar_checklist(df_listagem, df_checklist.copy(), df_fases)
df_resultado.head()


,CANAL,FILIAL,FILIAL_NOME,REGIAO,SKU,ARTIGO_COR,PRODUTO,DESC_PRODUTO,COR,TAM,...,Artigo Cor,Validação MINIMO,Validação RESSUPRIR,Franqueado,Pot. Venda,Sortimento Ajustado,NECESSIDADE ?,Validação NECESSIDADE,Filial-Artigo_Cor,fase
0,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-N10SI-M,H4FA-N10SI,H4FA,BERMUDA MASC,N10SI,M,...,H4FAN10SI,NÃO,SIM,NILO ARAUJO ARRUDA NETO,G,G_HA+HI,NÃO,SIM,411 - H4FA-N10SI,2ª Fase
1,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-XXG,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,XXG,...,H4FAAX7SI,SIM,SIM,NILO ARAUJO ARRUDA NETO,G,G_HA+HI,NÃO,SIM,411 - H4FA-AX7SI,2ª Fase
2,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-XG,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,XG,...,H4FAAX7SI,SIM,SIM,NILO ARAUJO ARRUDA NETO,G,G_HA+HI,NÃO,SIM,411 - H4FA-AX7SI,2ª Fase
3,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-P,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,P,...,H4FAAX7SI,SIM,SIM,NILO ARAUJO ARRUDA NETO,G,G_HA+HI,NÃO,SIM,411 - H4FA-AX7SI,2ª Fase
4,FRANQUIAS,411,HS SHOP PANTANAL,CENTRO-OESTE,H4FA-AX7SI-M,H4FA-AX7SI,H4FA,BERMUDA MASC,AX7SI,M,...,H4FAAX7SI,SIM,SIM,NILO ARAUJO ARRUDA NETO,G,G_HA+HI,NÃO,SIM,411 - H4FA-AX7SI,2ª Fase


In [26]:
df_resultado.columns

Index(['CANAL', 'FILIAL', 'FILIAL_NOME', 'REGIAO', 'SKU', 'ARTIGO_COR',
       'PRODUTO', 'DESC_PRODUTO', 'COR', 'TAM', 'TAM_GRADE', 'NEGOCIO',
       'PIRAMIDE', 'GENERO', 'IDADE_PRODUTO', 'GRUPO_PRODUTO', 'LINHA',
       'COLECAO', 'VELOCIDADE_VENDA', 'LEADTIME', 'COBERTURA', 'MINIMO',
       'PRESENTE', 'LOJA', 'TRANSITO', 'EMBALADO', 'RESERVADO', 'REGULADOR',
       'EM_MOVIMENTO', 'VENDAS_D_14', 'VENDAS_D_1', 'ALVO', 'NECESSIDADE',
       'RESSUPRIR', 'EST_TOTAL', 'EST_DISP', 'CATEGORIA_EST_TOTAL',
       'CATEGORIA_EST_DISP', 'DATA', 'CNPJ', 'CLISAP', 'CONT_RUPTURA',
       'CONT_RUPTURA_RESSUP', 'CONT_FALTA', 'VOLUME_FALTA', 'CONT_ALVO',
       'CONT_EXCESSO', 'VOLUME_EXCESSO', 'Alvo Calculado', 'Alvo Final',
       'Validação Alvo', 'Artigo Cor', 'Validação MINIMO',
       'Validação RESSUPRIR', 'Franqueado', 'Pot. Venda',
       'Sortimento Ajustado', 'NECESSIDADE ?', 'Validação NECESSIDADE',
       'Filial-Artigo_Cor', ' fase'],
      dtype='object')

In [27]:
# ... existing code ...
colunas_selecionadas = [
    'Alvo Calculado', 'Alvo Final', 'Artigo Cor', 'Validação MINIMO', 
    'Validação RESSUPRIR', 'Artigo Cor', 'Franqueado', 'Pot. Venda', 
    'Sortimento Ajustado', 'NECESSIDADE ?', 'Validação NECESSIDADE', 
    'Filial-Artigo_Cor', ' fase', 'Validação Alvo'
]

# Select the columns using a list
df_selecionado = df_resultado[colunas_selecionadas]

df_selecionado.isnull().sum()

Alvo Calculado           0
Alvo Final               0
Artigo Cor               0
Validação MINIMO         0
Validação RESSUPRIR      0
Artigo Cor               0
Franqueado               0
Pot. Venda               0
Sortimento Ajustado      0
NECESSIDADE ?            0
Validação NECESSIDADE    0
Filial-Artigo_Cor        0
 fase                    0
Validação Alvo           0
dtype: int64

In [28]:
filiais_sem_franqueado = df_resultado[df_resultado['Franqueado'].isnull()]['FILIAL']
print(filiais_sem_franqueado.unique())

[]


In [29]:
#exibir a coluna DATA da validação checklist
df_resultado['DATA'].head()


0    2025-12-17
1    2025-12-17
2    2025-12-17
3    2025-12-17
4    2025-12-17
Name: DATA, dtype: object

In [30]:
# Define o nome do arquivo e da aba de saída
nome_arquivo_saida = "(Validação) 38ª rodada_17.12.xlsx"
caminho_saida = output / nome_arquivo_saida
nome_aba = "Base rep."

print(f"Salvando resultado em '{nome_arquivo_saida}'...")

 #Salva o DataFrame no arquivo Excel especificado
df_resultado.to_excel(caminho_saida, sheet_name=nome_aba, index=False)

print(f"Processo concluído! Arquivo salvo em: {caminho_saida}")

Salvando resultado em '(Validação) 38ª rodada_17.12.xlsx'...
Processo concluído! Arquivo salvo em: OUTPUT\(Validação) 38ª rodada_17.12.xlsx
